<center><h1 style="color:#1a1a1a;
                    font-size:3em">
        NHANES - MTech Project 
        </h1> 
        <h2 style="color:#1a1a1a;
                    font-size:2em">
        Categorizing the data based on the Pre-Diabetic Risk test (Screening Tool) by ADA Guidelines 
            <h6> (https://www.cdc.gov/diabetes/prevention/pdf/Prediabetes-Risk-Test-Final.pdf)</h6>
       </h2>
       <h3 style="color:#1a1a1a;
                    font-size:2em">
        Prakash Easow Thomas - 2022DA04285 - June 2024
       </h3>
</center>

In [2]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import LabelEncoder

In [3]:
df1quest  = pd.read_csv('C:\Prakash\Personal\Studies\Techincal\MTech. Data Science and Engineering\Semester-wise\Semester-I\Study\Sem-I - 0- Python Fundamentals for Data Science (S1-22_DSECLPFDS)\PythonPgm\MTech-Proj03-NHANES-2018\Data\questionnaire.csv')
df2lab    = pd.read_csv('C:\Prakash\Personal\Studies\Techincal\MTech. Data Science and Engineering\Semester-wise\Semester-I\Study\Sem-I - 0- Python Fundamentals for Data Science (S1-22_DSECLPFDS)\PythonPgm\MTech-Proj03-NHANES-2018\Data\laboratory.csv')
df3exam   = pd.read_csv('C:\Prakash\Personal\Studies\Techincal\MTech. Data Science and Engineering\Semester-wise\Semester-I\Study\Sem-I - 0- Python Fundamentals for Data Science (S1-22_DSECLPFDS)\PythonPgm\MTech-Proj03-NHANES-2018\Data\examination.csv')
df4demo   = pd.read_csv('C:\Prakash\Personal\Studies\Techincal\MTech. Data Science and Engineering\Semester-wise\Semester-I\Study\Sem-I - 0- Python Fundamentals for Data Science (S1-22_DSECLPFDS)\PythonPgm\MTech-Proj03-NHANES-2018\Data\demographics.csv')


In [4]:
common_key = 'SEQN'
# Merge the DataFrames
merged_dfNHAENES2018 = df1quest.merge(df2lab, on=common_key, how='outer')
merged_dfNHAENES2018 = merged_dfNHAENES2018.merge(df3exam, on=common_key, how='outer')
merged_dfNHAENES2018 = merged_dfNHAENES2018.merge(df4demo, on=common_key, how='outer')


In [5]:
columns_to_select = ['SEQN',                                        # Sequence No
                     'RIDAGEYR',                                    # Age
                     'RIAGENDR',                                    # Gender
                     'RHQ162',                                      # Gestational Diabetes
                     'DIQ175A',                                     # Family History
                     'BPXOSY1','BPXODI1',                           # Systolic (>120) and Diastoilc > 80
                     'PAQ605','PAQ620','PAQ650','PAQ665', 'PAQ706', # Physically Active 
                     'BMXBMI',                                      # Body Mass Index
                     'BMXWT',                                       # Body Weight
                     'BMXHT',                                       # Height
                     'RIDRETH3',                                    # Race
                     'LBXGH',                                       # HbA1c
                     'LBXGLU'                                       # Fasting Blood Sugar
                     ]

In [6]:
# Ensure the columns to select are in the DataFrame
valid_columns = pd.Index(columns_to_select).intersection(merged_dfNHAENES2018.columns)
merged_dfNHAENES2018 = merged_dfNHAENES2018.drop(merged_dfNHAENES2018[merged_dfNHAENES2018['RIDRETH3'] !=  7].index)  # Others      
print(valid_columns)
# Select only the valid columns
merged_dfNHAENES2018_selected = merged_dfNHAENES2018[columns_to_select]

Index(['SEQN', 'RIDAGEYR', 'RIAGENDR', 'RHQ162', 'DIQ175A', 'BPXOSY1',
       'BPXODI1', 'PAQ605', 'PAQ620', 'PAQ650', 'PAQ665', 'PAQ706', 'BMXBMI',
       'BMXWT', 'BMXHT', 'RIDRETH3', 'LBXGH', 'LBXGLU'],
      dtype='object')


In [7]:
print(merged_dfNHAENES2018_selected)

          SEQN  RIDAGEYR  RIAGENDR  RHQ162  DIQ175A  BPXOSY1  BPXODI1  PAQ605  \
4      93707.0      13.0       1.0     NaN      NaN    136.0     71.0     NaN   
12     93715.0      71.0       1.0     NaN      NaN    126.0     78.0     1.0   
56     93759.0      60.0       1.0     NaN      NaN    136.0     78.0     1.0   
66     93769.0       6.0       1.0     NaN      NaN      NaN      NaN     NaN   
76     93779.0      10.0       2.0     NaN      NaN    107.0     56.0     NaN   
...        ...       ...       ...     ...      ...      ...      ...     ...   
9151  102854.0      45.0       1.0     NaN     10.0    131.0     81.0     1.0   
9208  102911.0      55.0       2.0     2.0     10.0    135.0     70.0     2.0   
9223  102926.0      15.0       2.0     NaN      NaN    107.0     61.0     NaN   
9233  102936.0       1.0       1.0     NaN      NaN      NaN      NaN     NaN   
9236  102939.0       2.0       2.0     NaN      NaN      NaN      NaN     NaN   

      PAQ620  PAQ650  PAQ66

In [8]:
def clean_data(df):
    for column in df.columns:
        # Check if the column contains byte-strings
        if df[column].dtype == object:
            # Decode byte-strings to regular strings
            df[column] = df[column].apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
        # Convert non-numeric values to NaN for numeric columns
        df[column] = pd.to_numeric(df[column], errors='ignore')
    return df

In [9]:
merged_dfNHAENES2018_Cleaned =clean_data(merged_dfNHAENES2018_selected)
print(merged_dfNHAENES2018_Cleaned)

          SEQN  RIDAGEYR  RIAGENDR  RHQ162  DIQ175A  BPXOSY1  BPXODI1  PAQ605  \
4      93707.0      13.0       1.0     NaN      NaN    136.0     71.0     NaN   
12     93715.0      71.0       1.0     NaN      NaN    126.0     78.0     1.0   
56     93759.0      60.0       1.0     NaN      NaN    136.0     78.0     1.0   
66     93769.0       6.0       1.0     NaN      NaN      NaN      NaN     NaN   
76     93779.0      10.0       2.0     NaN      NaN    107.0     56.0     NaN   
...        ...       ...       ...     ...      ...      ...      ...     ...   
9151  102854.0      45.0       1.0     NaN     10.0    131.0     81.0     1.0   
9208  102911.0      55.0       2.0     2.0     10.0    135.0     70.0     2.0   
9223  102926.0      15.0       2.0     NaN      NaN    107.0     61.0     NaN   
9233  102936.0       1.0       1.0     NaN      NaN      NaN      NaN     NaN   
9236  102939.0       2.0       2.0     NaN      NaN      NaN      NaN     NaN   

      PAQ620  PAQ650  PAQ66

C:\Users\praka\AppData\Local\Temp\ipykernel_32780\3940003188.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = pd.to_numeric(df[column], errors='ignore')


In [10]:
merged_dfNHAENES2018_Cleaned = merged_dfNHAENES2018_Cleaned.select_dtypes(include=[int, float])
column_to_check_HbA1c ='LBXGH'
merged_dfNHAENES2018_Cleaned =merged_dfNHAENES2018_Cleaned.dropna(subset=[column_to_check_HbA1c])
column_to_check_BMI = 'BMXBMI'
merged_dfNHAENES2018_Cleaned =merged_dfNHAENES2018_Cleaned.dropna(subset=[column_to_check_BMI])

merged_dfNHAENES2018_Cleaned =merged_dfNHAENES2018_Cleaned.reset_index(drop=True)

#dataset_for_ML = dataset_for_ML.reset_index(drop=True)
print(merged_dfNHAENES2018_Cleaned)

         SEQN  RIDAGEYR  RIAGENDR  RHQ162  DIQ175A  BPXOSY1  BPXODI1  PAQ605  \
0     93707.0      13.0       1.0     NaN      NaN    136.0     71.0     NaN   
1     93715.0      71.0       1.0     NaN      NaN    126.0     78.0     1.0   
2     93759.0      60.0       1.0     NaN      NaN    136.0     78.0     1.0   
3     93780.0      46.0       2.0     2.0      NaN    120.0     80.0     2.0   
4     93791.0      20.0       1.0     NaN      NaN    133.0     87.0     1.0   
..        ...       ...       ...     ...      ...      ...      ...     ...   
334  102695.0      14.0       2.0     NaN      NaN     99.0     58.0     NaN   
335  102838.0      41.0       1.0     NaN      NaN    132.0     86.0     2.0   
336  102854.0      45.0       1.0     NaN     10.0    131.0     81.0     1.0   
337  102911.0      55.0       2.0     2.0     10.0    135.0     70.0     2.0   
338  102926.0      15.0       2.0     NaN      NaN    107.0     61.0     NaN   

     PAQ620  PAQ650  PAQ665  PAQ706  BM

In [11]:
# Processing Age
merged_dfNHAENES2018_Cleaned= merged_dfNHAENES2018_Cleaned.dropna(subset=['RIDAGEYR'])
def classify(row):
    if  row['RIDAGEYR'] > 0 and row['RIDAGEYR'] < 40 :
        return 0
    elif row['RIDAGEYR'] >= 40 and row['RIDAGEYR'] < 50:
        return 1
    elif row['RIDAGEYR'] >= 50 and row['RIDAGEYR'] < 60:
        return 1
    elif row['RIDAGEYR'] >= 60:
        return 2
    else:
        return 99
merged_dfNHAENES2018_Cleaned['Age_point']  = merged_dfNHAENES2018_Cleaned.apply(lambda row: classify(row), axis=1)
merged_dfNHAENES2018_Cleaned.shape

(339, 19)

In [12]:
merged_dfNHAENES2018_Cleaned= merged_dfNHAENES2018_Cleaned[merged_dfNHAENES2018_Cleaned['Age_point']!= 99] 
merged_dfNHAENES2018_Cleaned.shape

(339, 19)

In [13]:
# Processing Gender
merged_dfNHAENES2018_Cleaned= merged_dfNHAENES2018_Cleaned.dropna(subset=['RIAGENDR'])
def classify(row):
    if row['RIAGENDR']   == 1 :
        return 1
    elif row['RIAGENDR'] == 2:
        return 0
    else:
        return 99
merged_dfNHAENES2018_Cleaned['Gender_point']  = merged_dfNHAENES2018_Cleaned.apply(lambda row: classify(row), axis=1)
merged_dfNHAENES2018_Cleaned= merged_dfNHAENES2018_Cleaned[merged_dfNHAENES2018_Cleaned['Gender_point']!= 99] 
merged_dfNHAENES2018_Cleaned.shape

(339, 20)

In [14]:
# Processing Family History

#merged_dfNHAENES2018_Cleaned= merged_dfNHAENES2018_Cleaned.dropna(subset=['DIQ175A'])
def classify(row):
    if row['DIQ175A']   == 10 :
        return 1
    else:
        return 0
merged_dfNHAENES2018_Cleaned['Family_History_point']  = merged_dfNHAENES2018_Cleaned.apply(lambda row: classify(row), axis=1)
merged_dfNHAENES2018_Cleaned.shape

(339, 21)

In [15]:
# Processing Blood Pressure  if age is 12 
# merged_dfNHAENES2018_Cleaned= merged_dfNHAENES2018_Cleaned.dropna(subset=['BPXOSY1','BPXODI1'])
#merged_dfNHAENES2018_Cleaned.shape

In [16]:
def classify(row):
    if  row['RIDAGEYR'] <= 30:
        return 0 
    elif row['BPXOSY1'] >= 125 or row['BPXODI1'] >= 85:
        return 1
    elif row['BPXOSY1'] < 125 and row['BPXODI1'] < 85:
        return 0
    else:
        return 1
merged_dfNHAENES2018_Cleaned['Diagnosed_BP_point']  = merged_dfNHAENES2018_Cleaned.apply(lambda row: classify(row), axis=1)
merged_dfNHAENES2018_Cleaned.shape

(339, 22)

In [17]:
merged_dfNHAENES2018_Cleaned= merged_dfNHAENES2018_Cleaned[merged_dfNHAENES2018_Cleaned['Diagnosed_BP_point']!= 99] 
merged_dfNHAENES2018_Cleaned.shape

(339, 22)

In [18]:
# Processing Physically Active cases
def check_valid_column(row):    
    if pd.notna(row['PAQ706']) >=3 and pd.notna(row['PAQ706'])  <= 7 and row['RIDAGEYR'] <= 17:
        return 0                                      
    elif (pd.notna(row['PAQ605']) == 1 or pd.notna(row['PAQ620'])  == 1 or pd.notna(row['PAQ650']) == 1 or pd.notna(row['PAQ665']) == 1) and  row['RIDAGEYR'] > 17 :
        return  0
    else:
        return 1
merged_dfNHAENES2018_Cleaned['Physically_Active_point']  = merged_dfNHAENES2018_Cleaned.apply(lambda row: check_valid_column(row), axis=1)
merged_dfNHAENES2018_Cleaned.shape

(339, 23)

In [19]:
merged_dfNHAENES2018_Cleaned= merged_dfNHAENES2018_Cleaned[merged_dfNHAENES2018_Cleaned['Physically_Active_point']!= 99] 
merged_dfNHAENES2018_Cleaned.shape

(339, 23)

In [20]:
# Processing Body mass Index
merged_dfNHAENES2018_Cleaned= merged_dfNHAENES2018_Cleaned.dropna(subset=['BMXBMI'])
merged_dfNHAENES2018_Cleaned.shape

(339, 23)

In [21]:
def classify(row):
    if row['BMXBMI']   < 25 :
        return 0
    elif row['BMXBMI'] >= 25 and row['BMXBMI'] < 30:
        return 1
    elif row['BMXBMI'] >= 30 and row['BMXBMI'] < 40:
        return 2
    elif row['BMXBMI'] >= 40:  
        return 3
    else:
        return 99
merged_dfNHAENES2018_Cleaned['BMI_Point']  = merged_dfNHAENES2018_Cleaned.apply(lambda row: classify(row), axis=1)
merged_dfNHAENES2018_Cleaned.shape

(339, 24)

In [22]:
merged_dfNHAENES2018_Cleaned= merged_dfNHAENES2018_Cleaned[merged_dfNHAENES2018_Cleaned['BMI_Point']!= 99] 
merged_dfNHAENES2018_Cleaned.shape

(339, 24)

In [23]:
# Processing Gestational Diabetes for Women
def classify(row):
    if row['Gender_point']   == 0 and (row['RHQ162'] == 1  or row['RHQ162'] == 3) :
        return 1
    else:
        return 0
merged_dfNHAENES2018_Cleaned['Gestational_Diabetes_point']  = merged_dfNHAENES2018_Cleaned.apply(lambda row: classify(row), axis=1)
merged_dfNHAENES2018_Cleaned.shape

(339, 25)

In [24]:
merged_dfNHAENES2018_Cleaned= merged_dfNHAENES2018_Cleaned[merged_dfNHAENES2018_Cleaned['LBXGH'].notna()] 
merged_dfNHAENES2018_Cleaned.shape

(339, 25)

In [25]:
merged_dfNHAENES2018_Cleaned

,SEQN,RIDAGEYR,RIAGENDR,RHQ162,DIQ175A,BPXOSY1,BPXODI1,PAQ605,PAQ620,PAQ650,...,RIDRETH3,LBXGH,LBXGLU,Age_point,Gender_point,Family_History_point,Diagnosed_BP_point,Physically_Active_point,BMI_Point,Gestational_Diabetes_point
0,93707.0,13.0,1.0,NaN,NaN,136.0,71.0,NaN,NaN,NaN,...,7.0,5.6,NaN,0,1,0,0,1,0,0
1,93715.0,71.0,1.0,NaN,NaN,126.0,78.0,1.0,1.0,1.0,...,7.0,6.2,NaN,2,1,0,1,0,0,0
2,93759.0,60.0,1.0,NaN,NaN,136.0,78.0,1.0,2.0,1.0,...,7.0,6.8,73.0,2,1,0,1,0,1,0
3,93780.0,46.0,2.0,2.0,NaN,120.0,80.0,2.0,2.0,2.0,...,7.0,5.3,109.0,1,0,0,0,0,2,0
4,93791.0,20.0,1.0,NaN,NaN,133.0,87.0,1.0,1.0,1.0,...,7.0,5.1,NaN,0,1,0,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,102695.0,14.0,2.0,NaN,NaN,99.0,58.0,NaN,NaN,NaN,...,7.0,5.6,92.0,0,0,0,0,1,0,0
335,102838.0,41.0,1.0,NaN,NaN,132.0,86.0,2.0,2.0,2.0,...,7.0,5.4,101.0,1,1,0,1,0,2,0
336,102854.0,45.0,1.0,NaN,10.0,131.0,81.0,1.0,1.0,2.0,...,7.0,6.4,NaN,1,1,1,1,0,3,0
337,102911.0,55.0,2.0,2.0,10.0,135.0,70.0,2.0,2.0,2.0,...,7.0,5.8,NaN,1,0,1,1,0,0,0


In [26]:
# Processing Calculating the Score and categorizing to high and low risk
merged_dfNHAENES2018_Cleaned['Tool_Total_points'] = merged_dfNHAENES2018_Cleaned[['Age_point' ,
                                                                            'Gender_point',
                                                                            'Family_History_point',
                                                                            'Diagnosed_BP_point',
                                                                            'Physically_Active_point',
                                                                            'BMI_Point', 
                                                                            'Gestational_Diabetes_point']].sum(axis=1)
def classify(row):
    if row['Tool_Total_points']   < 5 :
        return "Low Risk"
    else:
        return "High Risk"
merged_dfNHAENES2018_Cleaned['Tool_Risk_Category'] = merged_dfNHAENES2018_Cleaned.apply(lambda row: classify(row), axis=1)

In [27]:
#print(merged_dfNHAENES2018_Cleaned)

In [28]:
def classify(row):
    if row['LBXGH']     < 5.7 :
        return 0
    elif row ['LBXGH'] >= 5.7:
        return 1
#print(dataset_for_ML['LBXGH'])
merged_dfNHAENES2018_Cleaned['HbA1c_Risk']=  merged_dfNHAENES2018_Cleaned.apply(lambda row: classify(row), axis=1)

In [29]:
def classify(row):
    if row['HbA1c_Risk']   == 0 :
        return "Low Risk"
    else:
        return "High Risk"
merged_dfNHAENES2018_Cleaned['HbA1c_Risk_Category'] = merged_dfNHAENES2018_Cleaned.apply(lambda row: classify(row), axis=1)

In [30]:
print(merged_dfNHAENES2018_Cleaned)

         SEQN  RIDAGEYR  RIAGENDR  RHQ162  DIQ175A  BPXOSY1  BPXODI1  PAQ605  \
0     93707.0      13.0       1.0     NaN      NaN    136.0     71.0     NaN   
1     93715.0      71.0       1.0     NaN      NaN    126.0     78.0     1.0   
2     93759.0      60.0       1.0     NaN      NaN    136.0     78.0     1.0   
3     93780.0      46.0       2.0     2.0      NaN    120.0     80.0     2.0   
4     93791.0      20.0       1.0     NaN      NaN    133.0     87.0     1.0   
..        ...       ...       ...     ...      ...      ...      ...     ...   
334  102695.0      14.0       2.0     NaN      NaN     99.0     58.0     NaN   
335  102838.0      41.0       1.0     NaN      NaN    132.0     86.0     2.0   
336  102854.0      45.0       1.0     NaN     10.0    131.0     81.0     1.0   
337  102911.0      55.0       2.0     2.0     10.0    135.0     70.0     2.0   
338  102926.0      15.0       2.0     NaN      NaN    107.0     61.0     NaN   

     PAQ620  PAQ650  ...  Gender_point 

In [31]:
# Save the merged DataFrame to a new CSV file
merged_dfNHAENES2018.to_csv('C:\Prakash\Personal\Studies\Techincal\MTech. Data Science and Engineering\Semester-wise\Semester-I\Study\Sem-I - 0- Python Fundamentals for Data Science (S1-22_DSECLPFDS)\PythonPgm\MTech-Proj03-NHANES-2018\Data\After_Screening_Tool_Data.csv', index=False)

In [32]:
# Calculating and Displaying the counts tool data alone
Tool_High_Risk_condition = merged_dfNHAENES2018_Cleaned['Tool_Total_points'] >= 5
Tool_High_Risk_count= Tool_High_Risk_condition.sum()
Tool_Low_Risk_condition = merged_dfNHAENES2018_Cleaned['Tool_Total_points']  < 5
Tool_Low_Risk_count= Tool_Low_Risk_condition.sum()
print("Out of the sample we have "+ str(Tool_High_Risk_count) +" Tool High risk category and " +str(Tool_Low_Risk_count) + " Tool Low Risk Category")

Out of the sample we have 78 Tool High risk category and 261 Tool Low Risk Category


In [33]:
# Calculating and Displaying the counts HbA1c data alone
HbA1c_High_Risk_condition = merged_dfNHAENES2018_Cleaned['HbA1c_Risk'] == 1
HbA1c_High_Risk_count= HbA1c_High_Risk_condition.sum()
HbA1c_Low_Risk_condition = merged_dfNHAENES2018_Cleaned['HbA1c_Risk']  == 0
HbA1c_Low_Risk_count= HbA1c_Low_Risk_condition.sum()
print("Out of the sample we have "+ str(HbA1c_High_Risk_count) +" High High risk category and " +str(HbA1c_Low_Risk_count) + " High Low Risk Category")

Out of the sample we have 115 High High risk category and 224 High Low Risk Category


In [34]:
# Classifying the TP, TN, FP, and FN.
TP =((merged_dfNHAENES2018_Cleaned['HbA1c_Risk'] == 1) & (merged_dfNHAENES2018_Cleaned['Tool_Total_points'] >= 5)).sum()
TN =((merged_dfNHAENES2018_Cleaned['HbA1c_Risk'] == 0) & (merged_dfNHAENES2018_Cleaned['Tool_Total_points'] <  5)).sum()
FP =((merged_dfNHAENES2018_Cleaned['HbA1c_Risk'] == 0) & (merged_dfNHAENES2018_Cleaned['Tool_Total_points'] >= 5)).sum()
FN =((merged_dfNHAENES2018_Cleaned['HbA1c_Risk'] == 1) & (merged_dfNHAENES2018_Cleaned['Tool_Total_points'] <  5)).sum()


In [35]:
print(f"True Positives  (TP): {TP}")
print(f"True Negatives  (TN): {TN}")
print(f"False Positives (FP): {FP}")
print(f"False Negatives (FN): {FN}")

True Positives  (TP): 57
True Negatives  (TN): 203
False Positives (FP): 21
False Negatives (FN): 58


In [36]:
Accuracy = (TP + TN) / (TP + TN + FP + FN) 
Precision = TP / (TP + FP)     
Recall = TP / (TP + FN)  
Specificity =TN / (TN + FP) 
F1_Score = 2 * (Precision * Recall) / (Precision + Recall)                                              

In [37]:
print(f'Accuracy    : {Accuracy:.4f}')
print(f'Precision   : {Precision:.4f}')
print(f'Recall      : {Recall:.4f}')
print(f'Specificity : {Specificity:.4f}')
print(f'F1 Score    : {F1_Score:.4f}')

Accuracy    : 0.7670
Precision   : 0.7308
Recall      : 0.4957
Specificity : 0.9062
F1 Score    : 0.5907
